In [2]:
# LSTM for sales problem with regression framing

import numpy
import matplotlib.pyplot as plt
import pandas
import math
import pickle
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

def create_dataset(dataset, look_back=1):
    dataX, dataY = [], []
    for i in range(len(dataset)-look_back-1):
        a = []
        for j in range(look_back):
            a = numpy.concatenate([a, dataset[i + j][:-1]])
        print(a)
        dataX.append(a)
        dataY.append(dataset[i + look_back, -1])
    return numpy.array(dataX), numpy.array(dataY)

# fix random seed for reproducibility
numpy.random.seed(7)

# load the dataset
dataframe = pandas.read_csv('data/Nl Pb till Dec 2016_ol-10_userBase-10_training-set_filled.csv', engine='python', skipfooter=0)

In [4]:
dataset = dataframe.values
dataset = dataset.astype('float32')

In [114]:
# normalize the dataset
#scaler = MinMaxScaler(feature_range=(0, 1))
#dataset = scaler.fit_transform(dataset)

In [5]:
# split into train and test sets
train_size = int(len(dataset) * 0.67)
test_size = len(dataset) - train_size
train, test = dataset[0:train_size,:], dataset[train_size:len(dataset),:]

In [ ]:
# reshape into X=t and Y=t+1
look_back = 1
trainX, trainY = create_dataset(train, look_back)
testX, testY = create_dataset(test, look_back)

In [174]:
# reshape input to be [samples, time steps, features]
#trainX = numpy.reshape(trainX, (trainX.shape[0], trainX.shape[2], look_back))
#trainX = numpy.reshape(trainX, (trainX.shape[0] * trainX.shape[1], look_back))

#testX = numpy.reshape(testX, (testX.shape[0] * testX.shape[1], look_back))

In [7]:
# create and fit the LSTM network
model = Sequential()
model.add(Dense(40, input_dim=look_back*32, activation='relu'))
model.add(Dense(40,  activation='relu'))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(trainX, trainY, nb_epoch=100, batch_size=5, verbose=2)

Epoch 1/100
0s - loss: 3236552.3882
Epoch 2/100
0s - loss: 2458572.6947
Epoch 3/100
0s - loss: 1098660.0590
Epoch 4/100
0s - loss: 1295324.0943
Epoch 5/100
0s - loss: 1442314.9427
Epoch 6/100
0s - loss: 1140651.4923
Epoch 7/100
0s - loss: 1102930.6728
Epoch 8/100
0s - loss: 1256910.9060
Epoch 9/100
0s - loss: 797449.6480
Epoch 10/100
0s - loss: 1236313.4149
Epoch 11/100
0s - loss: 892428.9022
Epoch 12/100
0s - loss: 648314.4823
Epoch 13/100
0s - loss: 885374.7016
Epoch 14/100
0s - loss: 752839.2761
Epoch 15/100
0s - loss: 611461.9205
Epoch 16/100
0s - loss: 861423.1988
Epoch 17/100
0s - loss: 599829.7638
Epoch 18/100
0s - loss: 687636.7367
Epoch 19/100
0s - loss: 691331.3966
Epoch 20/100
0s - loss: 585959.5643
Epoch 21/100
0s - loss: 550876.1534
Epoch 22/100
0s - loss: 526937.9390
Epoch 23/100
0s - loss: 521301.4066
Epoch 24/100
0s - loss: 534662.7145
Epoch 25/100
0s - loss: 471353.0396
Epoch 26/100
0s - loss: 508400.9424
Epoch 27/100
0s - loss: 561270.8627
Epoch 28/100
0s - loss: 5298

In [7]:
import sys
sys.setrecursionlimit(10000)
# Save the model
# pickle.dump( model, open( "save.p", "wb" ) )

In [8]:
# make predictions
trainPredict = model.predict(trainX)
testPredict = model.predict(testX)

In [9]:
testScore = math.sqrt(mean_squared_error(testY, testPredict[:,0]))
print('Test Score: %.2f RMSE' % (testScore))

Test Score: 796.18 RMSE
